In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
!pip install py_vncorenlp
!pip install transformers[sentencepiece]
!pip install accelerate -U
!pip install evaluate
!pip install sacrebleu

  Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)


In [ ]:
!git clone https://github.com/feralvam/easse.git && cd easse && pip install .

Cloning into 'easse'...
remote: Enumerating objects: 1964, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 1964 (delta 118), reused 104 (delta 104), pack-reused 1819
Receiving objects: 100% (1964/1964), 33.15 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (1231/1231), done.
Processing /content/easse
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-wyb12ygi/tseval_4cb4d89662d449e08da20ce4095afe88
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-wyb12ygi/tseval_4cb4d89662d449e08da20ce4095afe88
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit dea8863683ea5946fd50184883c9be7a7339e821
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/8

In [ ]:
from easse.sari import corpus_sari

def sentence_sari(orig_sent: str, sys_sent: str, ref_sents:list[str]):
  return corpus_sari(orig_sents=[orig_sent],
                sys_sents=[sys_sent],
                refs_sents=[ref_sents])
sentence_sari("Tôi học lớp 5 .", "Tôi học_sinh lớp 5.", ["Tôi là học_sinh lớp 5 ."])

69.58333333333333

In [ ]:
import evaluate
sacrebleu = evaluate.load("sacrebleu")

In [ ]:
predictions = ["Tôi là học lớp 5 ."]
references = [["Tôi là học_sinh lớp 5 ."]]
def sentence_bleu(prediction:str, refs:list[str]):
  return sacrebleu.compute(predictions=[prediction], references=[refs], tokenize="none")['score']
sentence_bleu("Tôi học_sinh lớp 5 .", ["Tôi là học_sinh lớp 5 ."])

57.89300674674101

In [ ]:
sentence_sari("Trong truyền_thống diễn_xướng sử_thi hình_thái sống của Trung_Quốc hôm_nay không thiếu những cá_nhân vĩ_đại như_vậy, ví_dụ như Trát_Ba, Tang_Châu của dân_tộc Tạng, nghệ_nhân Ma_Ma_Yi của dân_tộc Kirgiz, nghệ_nhân Ba Kiệt, Kim_Ba_Trát_Mộc Tô, Chu_Nãi và P.Arimpil của dân_tộc Mông_Cổ, nghệ_nhân Khúc_Mạc_Y_Nặc của dân_tộc Di...",
              "Trong truyền_thống diễn_xướng sử_thi hình_thái sống của Trung_Quốc hôm_nay không thiếu những cá_nhân vĩ_đại như_vậy, ví_dụ như Trát_Ba, Tang_Châu của dân_tộc Tạng, nghệ_nhân Ma_Ma_Yi của dân_tộc Kirgiz, nghệ_nhân Ba Kiệt, Kim_Ba_Trát_Mộc Tô, Chu_Nãi và P.Arimpil của dân_tộc Mông_Cổ, nghệ_nhân Khúc_Mạc_Y_Nặc của dân_tộc Di...",
              ["Trong truyền_thống kể chuyện lịch_sử của Trung_Quốc hiện_đại, có nhiều nhân_vật lớn như Trát_Ba, Tang_Châu của người Tạng, nghệ_nhân Ma_Ma_Yi của người Kirgiz, Ba Kiệt, Kim_Ba_Trát_Mộc Tô, Chu_Nãi và P.Arimpil của người Mông_Cổ, nghệ_nhân Khúc_Mạc_Y_Nặc của người Di..."]
)

22.941346129599566

In [ ]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local working folder
py_vncorenlp.download_model(save_dir='/content')

In [ ]:
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

In [ ]:
from datasets import load_dataset

data_files = {
    "train": "train.json",
    "eval": "eval.json"
}

dataset = load_dataset("/content/drive/MyDrive/sentence-simplification-nckh/dataset", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['result', 'id', 'sentence'],
        num_rows: 9160
    })
    eval: Dataset({
        features: ['result', 'id', 'sentence'],
        num_rows: 500
    })
})

In [ ]:
dataset['train'][0]

{'result': 'Mọi người đều biết, Soneca và người châu Âu tạo ra "định mệnh" và "thảm họa". Đối với người Hy Lạp, "thảm họa" chỉ là một sự thay đổi, một sự lật ngược. Nói cách khác, nó chỉ là một sự chuyển động vòng tròn, sau đó nó sẽ quay trở lại.',
 'id': 'difficult_102_5',
 'sentence': 'Ai cũng biết , " định mệnh " và " thảm họa " là do Soneca và những người châu Âu bịa ra ; với những người Hy Lạp , " thảm họa " là bước ngoặt quay ngược lại , đảo xuống phía dưới , dẫu vậy , nó cũng chỉ là " bước ngoặt " , có nghĩa , nó là sự chuyển động theo vòng tròn , tới lượt sau nó lại đảo lên phía trên .'}

# Preprocess

In [ ]:
dict_map = {
    "òa": "oà",
    "Òa": "Oà",
    "ÒA": "OÀ",
    "óa": "oá",
    "Óa": "Oá",
    "ÓA": "OÁ",
    "ỏa": "oả",
    "Ỏa": "Oả",
    "ỎA": "OẢ",
    "õa": "oã",
    "Õa": "Oã",
    "ÕA": "OÃ",
    "ọa": "oạ",
    "Ọa": "Oạ",
    "ỌA": "OẠ",
    "òe": "oè",
    "Òe": "Oè",
    "ÒE": "OÈ",
    "óe": "oé",
    "Óe": "Oé",
    "ÓE": "OÉ",
    "ỏe": "oẻ",
    "Ỏe": "Oẻ",
    "ỎE": "OẺ",
    "õe": "oẽ",
    "Õe": "Oẽ",
    "ÕE": "OẼ",
    "ọe": "oẹ",
    "Ọe": "Oẹ",
    "ỌE": "OẸ",
    "ùy": "uỳ",
    "Ùy": "Uỳ",
    "ÙY": "UỲ",
    "úy": "uý",
    "Úy": "Uý",
    "ÚY": "UÝ",
    "ủy": "uỷ",
    "Ủy": "Uỷ",
    "ỦY": "UỶ",
    "ũy": "uỹ",
    "Ũy": "Uỹ",
    "ŨY": "UỸ",
    "ụy": "uỵ",
    "Ụy": "Uỵ",
    "ỤY": "UỴ",
    }

def replace_all(text, dict_map):
    for i, j in dict_map.items():
        text = text.replace(i, j)
    return text

In [ ]:
rdrsegmenter.word_segment("Hãy làm câu sau đây trở nên đơn giản hơn : ")

['Hãy làm câu sau đây trở_nên đơn_giản hơn :']

In [ ]:
def process_text(example):
  example["sentence"] = example["sentence"].strip()
  example["sentence"] = replace_all(example["sentence"], dict_map)
  example["result"] = example["result"].strip()
  example["result"] = replace_all(example["result"], dict_map)

def segment(example):
  example["sentence"] = " ".join(rdrsegmenter.word_segment(example["sentence"]))
  example["result"] = " ".join(rdrsegmenter.word_segment(example["result"]))
  return example

# def add_prefix(example):
#   example["sentence"] = "Hãy làm câu sau đây trở_nên đơn_giản hơn : " + example["sentence"]
#   return example

In [ ]:
dataset = dataset.map(process_text)
dataset = dataset.map(segment)
# dataset["eval"] = dataset["eval"].map(add_prefix)

Map:   0%|          | 0/9160 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Parameter 'function'=<function segment at 0x7941f06f3250> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/9160 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
dataset["eval"][0]

{'result': 'Trong truyền_thống kể chuyện lịch_sử của Trung_Quốc hiện_đại , có nhiều nhân_vật lớn như Trát_Ba , Tang_Châu của người Tạng , nghệ_nhân Ma_Ma_Yi của người Kirgiz , Ba Kiệt , Kim_Ba_Trát_Mộc Tô , Chu_Nãi và P.Arimpil của người Mông_Cổ , nghệ_nhân Khúc_Mạc_Y_Nặc của người Di ...',
 'id': 'difficult_084_63',
 'sentence': 'Trong truyền_thống diễn_xướng sử_thi hình_thái sống của Trung_Quốc hôm_nay không thiếu những cá_nhân vĩ_đại như_vậy , ví_dụ như Trát_Ba , Tang_Châu của dân_tộc Tạng , nghệ_nhân Ma_Ma_Yi của dân_tộc Kirgiz , nghệ_nhân Ba Kiệt , Kim_Ba_Trát_Mộc Tô , Chu_Nãi và P.Arimpil của dân_tộc Mông_Cổ , nghệ_nhân Khúc_Mạc_Y_Nặc của dân_tộc Di ...'}

In [ ]:
dataset["train"][1]

{'result': 'Các khó_khăn như vấn_đề tiền_bạc , áp_lực học_hành , cảm_giác cô_đơn và lối sống mới đã giúp họ trưởng_thành . Nhưng quan_trọng hơn , trong những rắc_rối ấy , tình_yêu dịu_ngọt vẫn xuất_hiện , giúp họ tin vào những điều tốt_đẹp trong cuộc_sống .',
 'id': 'difficult_085_63',
 'sentence': 'Những va_đập đầy khắc_nghiệt với cuộc_sống mới như các vấn_đề tiền_bạc , rồi áp_lực học_hành , sự cô_độc cùng với lối sinh_hoạt khác lạ , ... đều khiến họ trưởng_thành , nhưng quan_trọng hơn , trong những bộn_bề ấy , những thương_yêu dịu_ngọt vẫn xuất_hiện theo một_cách nào đó , giúp họ thêm tin_yêu vào những điều tốt_đẹp ở trên đời .'}

Prepare for training

In [ ]:
from transformers import AutoTokenizer, MBartForConditionalGeneration, AutoConfig, Seq2SeqTrainingArguments, Seq2SeqTrainer, GenerationConfig
import torch
# from bartpho_dataset import dataset
from tqdm import tqdm
import os

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model_name = "vinai/bartpho-word"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
model = MBartForConditionalGeneration.from_pretrained(model_name, config = config).to(device)

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
model.generation_config.max_length = 300
model.generation_config.num_beams = 4
model.generation_config.do_sample = False
model.generation_config

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 300,
  "num_beams": 4,
  "pad_token_id": 1
}

# Test generation

In [ ]:
# print(dataset["train"][0]["result"])
# arr = tokenizer("mọi người đều biết , soneca và người châu_âu tạo ra \" định_mệnh \" và \" thảm_hoạ \" đối_với người hy_lạp , \" thảm_hoạ \" chỉ là một sự thay_đổi , một sự lật_ngược nói cách khác , nó chỉ là một sự chuyển_động vòng_tròn , sau đó nó sẽ quay trở_lại", truncation=True, return_tensors='pt')
# print(arr)

Mọi người đều biết , Soneca và người châu_Âu tạo ra " định_mệnh " và " thảm_hoạ " . Đối_với người Hy_Lạp , " thảm_hoạ " chỉ là một sự thay_đổi , một sự lật_ngược . Nói cách khác , nó chỉ là một sự chuyển_động vòng_tròn , sau đó nó sẽ quay trở_lại .
{'input_ids': tensor([[    0,   207,    18,   131,    55,     4, 12695,  1358,   853,     6,
            18, 26705,  8145,   199,    40,    22, 10611,    22,     6,    22,
          3561,    22,   190,    18, 57387, 34886,     4,    22,  3561,    22,
            66,     8,    16,    61,   411,     4,    16,    61, 19902,    96,
           139,    85,     4,   231,    66,     8,    16,    61,  6023, 10006,
             4,    53,    37,   231,    38,   588,   607,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
# tokenizer.decode(arr["input_ids"][0], skip_special_tokens=True)

'mọi người đều biết, soneca và người châu_âu tạo ra " định_mệnh " và " thảm_hoạ " đối_với người hy_lạp, " thảm_hoạ " chỉ là một sự thay_đổi, một sự lật_ngược nói cách khác, nó chỉ là một sự chuyển_động vòng_tròn, sau đó nó sẽ quay trở_lại'

In [ ]:
# result = model.generate(arr["input_ids"])[0]

In [ ]:
# tokenizer.decode(result)

'</s> <s> mọi người đều biết, soneca và người châu_âu tạo ra " định_mệnh " và " thảm_hoạ " đối_với người hy_lạp, " thảm_hoạ " chỉ là một sự thay_đổi, một sự lật_ngược nói cách khác, nó chỉ là một sự chuyển_động vòng_tròn, sau đó nó sẽ quay trở_lại </s>'

# Tokenize

In [ ]:
def tokenize(examples):
  tokenize_config = {
      "truncation": True,
      "max_length": 300,
      # "return_tensors": "pt"
  }
  model_inputs = tokenizer(text=examples["sentence"], **tokenize_config)
  model_inputs["labels"] = tokenizer(text_target=examples["result"], **tokenize_config)["input_ids"]
  return model_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize, remove_columns=["id", "sentence", "result"])
tokenized_dataset

Map:   0%|          | 0/9160 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9160
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels, inputs = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)
    word_sari_score = 0
    syllable_sari_score = 0
    bleu_score = 0
    for pred, label, input in zip(decoded_preds, decoded_labels, decoded_inputs):
      word_sari_score += sentence_sari(input, pred, [label])
      syllable_sari_score += sentence_sari(input.replace("_", ""), pred.replace("_", ""), [label.replace(" ", "_")])
      bleu_score += sentence_bleu(pred, [label])
    word_sari_score /= len(predictions)
    syllable_sari_score /= len(predictions)
    bleu_score /= len(predictions)

    return {
        "SARI_word": word_sari_score,
        "SARI_syllable": syllable_sari_score,
        "BLEU": bleu_score
    }

In [ ]:
from transformers import AdamW

num_train_epochs = 2
per_device_train_batch_size = 8
gradient_accumulation_steps = 8
batch_size = per_device_train_batch_size * gradient_accumulation_steps
training_steps = (int(len(dataset["train"]) / batch_size)) * num_train_epochs
print("Training step: ", training_steps)
print("Warmup step: ", int(training_steps*0.1))
optimizer = AdamW(params=model.parameters(), lr=3e-5, weight_decay=0.001)

Training step:  286
Warmup step:  28


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
int(3.9)

3

In [ ]:
from transformers import get_scheduler

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(training_steps*0.1),
    num_training_steps=training_steps,
)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/sentence-simplification-nckh/bart-word-checkpoint",
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    per_device_eval_batch_size = 16,
    save_total_limit = 4,
    save_strategy = "steps",
    evaluation_strategy = "steps",
    eval_steps = 36,
    save_steps = 36,
    gradient_accumulation_steps = gradient_accumulation_steps,
    load_best_model_at_end = True,
    logging_steps = 36,
    predict_with_generate=True,
    include_inputs_for_metrics=True,
    # fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
    # test_dataset=tokenized_dataset["eval"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler)
)

In [ ]:
trainer.evaluate()

{'eval_loss': 1.325745701789856,
 'eval_SARI_word': 48.28002701410341,
 'eval_SARI_syllable': 31.412898082904512,
 'eval_BLEU': 19.341221138781272,
 'eval_runtime': 78.1695,
 'eval_samples_per_second': 6.396,
 'eval_steps_per_second': 0.409,
 'epoch': 2.0}

In [ ]:
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/sentence-simplification-nckh/bart-word-checkpoint/checkpoint-252")

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Step,Training Loss,Validation Loss


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=286, training_loss=0.16304138823822661, metrics={'train_runtime': 165.019, 'train_samples_per_second': 111.017, 'train_steps_per_second': 1.733, 'total_flos': 4221805817757696.0, 'train_loss': 0.16304138823822661, 'epoch': 2.0})

In [ ]:
outputs = trainer.predict(tokenized_dataset["eval"])

In [ ]:
predictions = tokenizer.batch_decode(outputs.predictions, skip_special_tokens=True)

In [ ]:
dataset["eval"][0]

{'result': 'Trong truyền_thống kể chuyện lịch_sử của Trung_Quốc hiện_đại , có nhiều nhân_vật lớn như Trát_Ba , Tang_Châu của người Tạng , nghệ_nhân Ma_Ma_Yi của người Kirgiz , Ba Kiệt , Kim_Ba_Trát_Mộc Tô , Chu_Nãi và P.Arimpil của người Mông_Cổ , nghệ_nhân Khúc_Mạc_Y_Nặc của người Di ...',
 'id': 'difficult_084_63',
 'sentence': 'Trong truyền_thống diễn_xướng sử_thi hình_thái sống của Trung_Quốc hôm_nay không thiếu những cá_nhân vĩ_đại như_vậy , ví_dụ như Trát_Ba , Tang_Châu của dân_tộc Tạng , nghệ_nhân Ma_Ma_Yi của dân_tộc Kirgiz , nghệ_nhân Ba Kiệt , Kim_Ba_Trát_Mộc Tô , Chu_Nãi và P.Arimpil của dân_tộc Mông_Cổ , nghệ_nhân Khúc_Mạc_Y_Nặc của dân_tộc Di ...'}

In [ ]:
predictions[0]

'Trong truyền_thống diễn_xướng sử_thi của Trung_Quốc, có nhiều nghệ_nhân nổi_tiếng như Trát_Ba, Tang_Châu của Tạng, Ma_Ma_Yi của Kirgiz, Ba Kiệt, Kim_Ba_Trát_Mộc Tô, Chu_Nãi và P.Arimpil của Mông_Cổ, Khúc_Mạc_Y_Nặc của Di...'

In [ ]:
dataset["eval"][1]

{'result': 'Sau một trận đánh nhau , Ngọc và Bường đã hiểu rõ hơn về nhau và nhận ra rằng chỉ có sự nhẹ_nhàng , tình_thương và hoà_hợp mới giữ được mối quan_hệ giữa con_người và thế_giới .',
 'id': 'difficult_013_104',
 'sentence': 'Trong một đêm trăng , Ngọc và Bường sau khi đánh nhau chí_chết đã hiểu thêm về nhau , những minh_triết và đời thực , những chân_lý : " chỉ có nhẹ_nhàng , mềm_mỏng lịch_thiệp , tình_thương , hoà_hợp , yên_ổn ... mới khép_kín được quan_hệ giữa những con_người và quan_hệ giữa từng con_người với toàn thế_giới " ( Những người thợ xẻ ) .'}

In [ ]:
predictions[1]

'Trong một đêm trăng, Ngọc và Bường đã hiểu thêm về nhau, hiểu rõ về tính_cách và cuộc_sống thực, và nhận ra rằng chỉ có sự nhẹ_nhàng, lịch_thiệp, tình_thương, hoà_hợp, yên_ổn... mới kết_nối được mọi người và thế_giới.'

In [ ]:
import json

def load_json(file_path):
  with open(file_path, 'r', encoding='utf-8') as json_file:
    content = json.load(json_file)
  return content

def save_json(file_path, data):
  with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=2)

In [ ]:
result = []
for i, j in zip(dataset["eval"], predictions):
  temp = {**i}
  temp["pred"] = j
  result.append(temp)
result[0]

{'result': 'Trong truyền_thống kể chuyện lịch_sử của Trung_Quốc hiện_đại , có nhiều nhân_vật lớn như Trát_Ba , Tang_Châu của người Tạng , nghệ_nhân Ma_Ma_Yi của người Kirgiz , Ba Kiệt , Kim_Ba_Trát_Mộc Tô , Chu_Nãi và P.Arimpil của người Mông_Cổ , nghệ_nhân Khúc_Mạc_Y_Nặc của người Di ...',
 'id': 'difficult_084_63',
 'sentence': 'Trong truyền_thống diễn_xướng sử_thi hình_thái sống của Trung_Quốc hôm_nay không thiếu những cá_nhân vĩ_đại như_vậy , ví_dụ như Trát_Ba , Tang_Châu của dân_tộc Tạng , nghệ_nhân Ma_Ma_Yi của dân_tộc Kirgiz , nghệ_nhân Ba Kiệt , Kim_Ba_Trát_Mộc Tô , Chu_Nãi và P.Arimpil của dân_tộc Mông_Cổ , nghệ_nhân Khúc_Mạc_Y_Nặc của dân_tộc Di ...',
 'pred': 'Trong truyền_thống diễn_xướng sử_thi của Trung_Quốc, có nhiều nghệ_nhân nổi_tiếng như Trát_Ba, Tang_Châu của Tạng, Ma_Ma_Yi của Kirgiz, Ba Kiệt, Kim_Ba_Trát_Mộc Tô, Chu_Nãi và P.Arimpil của Mông_Cổ, Khúc_Mạc_Y_Nặc của Di...'}

In [ ]:
save_json("/content/drive/MyDrive/sentence-simplification-nckh/bart_2epochs.json", result)

In [ ]:
trainer.sa